In [2]:
# ------------------------------------------------------------------------------------------------------------------------- #
import sys

# sudoku class (constructors, getters, setters, and functions)

class sudoku():
    
    # object constructor
    def __init__(self, puzzle):
        
        # puzzle as list attribute (field)
        self.puzzle = [int(v) for v in puzzle]
        
        # copy of initial puzzle field
        self.copy = self.puzzle 
        
        # Initialize the 'unassigned' lists ? / dictionary == map?
        # "dictionary" of unassigned row field 
        # * see comments for getrow below
        
        self.unrow = {}            
        for i in range(9):
            self.unrow[i] = [x for x in list(range(1,10)) if x not in self.getrow(i)]

        self.uncol = {}            
        for i in range(9):
            self.uncol[i] = [x for x in list(range(1,10)) if x not in self.getcol(i)]

        self.unbox = {}            
        for i in range(9):
            self.unbox[i] = [x for x in list(range(1,10)) if x not in self.getbox(i)]

        # cell wise un assigned list
        self.uncell = {}
        for i in range(9):
            for j in range(9):
                idx = 9*i+j
                if (self.puzzle[idx]): # cell index pre occupied
                    pass
                else:
                    # i, j, k => row, col, box index
                    k = (idx - 27*(int(idx/27))) % 9 // 3  +  3*int(idx/27) 
                    self.uncell[idx] = list(set.intersection(set(self.unrow[i]), set(self.uncol[j]), set(self.unbox[k])))
                    # intersection of row / col / box unassigned list is the cell unassigned list
     
        # empty index field
        self.empty = [x for x in range(81) if (self.puzzle[x] == 0)]

        # nonempty index field / for checking later
        self.nonempty = [x for x in range(81) if (self.puzzle[x] != 0)]
        
# ------------------------------------------------------------------------------------------------------------------------- #
    
    # printing methods

    def __repr__(self):
        return "".join(str(x) for x in self.puzzle) + "\n"

    def __str__(self):
        s = "\n+---+---+---+"
        for b in range(3):
            for r in range(3):
                s += "\n|"
                for c in range(3):
                  s += "".join([str(v) for v in self.puzzle[b*27+r*9+c*3:b*27+r*9+c*3+3]]) + "|"
        s += "\n+---+---+---+"

        # print(s)
        return s

# ------------------------------------------------------------------------------------------------------------------------- #

    # made these ith row / col / box getter before looking at the detailed direction 
    # used as auxiliary functions in constructor, "index by cell" row / col / box getters, verify_solution
    # parameter : sudoku object, row / col / box index
    # return : ith row / col / box

    def getrow(self,i, num=True):
        row = [self.puzzle[i*9+x] for x in range(9)]
        if (num==True):
            row = list(map(int, row))
        return(row)

    def getcol(self,i, num=True):
        col = [self.puzzle[i+x*9] for x in range(9)]
        if (num==True):
            col = list(map(int, col))
        return(col)

    def getbox(self,i, num=True):
        box = [self.puzzle[(x%3) + (x//3)*9 + (i%3) * 3 + (i//3)*27] for x in range(9)]
        if (num==True):
            box = list(map(int, box))
        return(box)
   
    
    # "index by cell" row / col / box getters
    # parameter : sudoku object, cell index
    # return : row / col / box that the cell at the input index belongs to 

    # getrow(1 st row) == get_row(0,1,2,9,10,11,18,19,20th cell)
    # getbox(puzz,4) == get_box(puzz,31)
    # not used very often since getrow(indexing mapping e.x. i//9 ) == get_row(i)

    def get_row(self, cell):
        return(self.getrow(cell//9))

    def get_col(self, cell):
        return(self.getcol(cell%9))

    def get_box(self, cell):
        i = (cell - 27*(int(cell/27))) % 9 // 3  +  3*int(cell/27)
        return(self.getbox(i))

# ------------------------------------------------------------------------------------------------------------------------- #

    # auxiliary function for assign
    #  find intersection of row / col / box candidates (possibilites) 

    def cell_candidate(self, cell):

        # i-th cell => i//9 th row
        a = self.unrow[cell//9]
        # ith cell => i%9 th col
        b = self.uncol[cell%9]
        # ith cell => (cell - 27*(int(cell/27))) % 9 // 3  +  3*int(cell/27) th box
        c = self.unbox[ (cell - 27*(int(cell/27))) % 9 // 3  +  3*int(cell/27) ]
        v = set.intersection(set(a), set(b), set(c))

        return(list(v))

# ------------------------------------------------------------------------------------------------------------------------- #

    # assign single candidate to a cell
    # remove the cell index from list of empty indices
    # remove assignment from dictionary of unassigned row / col / box
    
    def assign_cell(self,i,cell_value):
        self.puzzle[i] = cell_value
        self.empty.remove(i) # remove empty index
        # remove
        self.unrow[i//9].remove(cell_value) # remove unassigned row
        self.uncol[i%9].remove(cell_value) # col
        self.unbox[ (i - 27*(int(i/27))) % 9 // 3  +  3*int(i/27) ].remove(cell_value) # box
        # del(self.uncell[i])
    
    
    def assign(self):
        for i in self.empty:
            po = self.cell_candidate(i)
            if len(po) == 1:
                self.assign_cell(i,po[0])

        
# initial assign (assign_cell + assign) 
# later seperated for recurse
    
#     def assign(self):
#         for i in self.empty:
#             pos = self.cell_candidate(i)
#             if len(pos) == 1:
#                 self.puzzle[i] = pos[0] # assign

#                 self.empty.remove(i) # remove empty index

#                 self.unrow[i // 9].remove(pos[0]) # remove unassigned row
#                 self.uncol[i % 9].remove(pos[0]) # col
#                 self.unbox[ (i - 27*(int(i/27))) % 9 // 3  +  3*int(i/27) ].remove(pos[0]) # box
#         pass

# ------------------------------------------------------------------------------------------------------------------------- #

    # solve methods : run assign() until puzzle is solved
    # parameters: sudoku object
    # return : none assign new values to empty cells

    def solve(self):
        
        # while (self.verify_solution() != True):
        #    self.assign()
        # pass
        
        ## but if can't be solved by "easy" solution need to stop at some point
        init = len(self.empty)
        self.assign()
        # print("change", init - len(self.empty))
        
        while (init - len(self.empty) != 0): # while converge
            init = len(self.empty)
            # print("left", init)
            self.assign()

# ------------------------------------------------------------------------------------------------------------------------- #

    # auxiliary functions for checking row / col / box (verify solution critieria 2)
    # parameter : sudoku object
    # return : True of False

    def checkrow(self):
        for i in range(1,10):
            # temp = getrow(self,i)
            # temp2 = sorted(temp)
            if (sorted(self.getrow(i)) != list(range(1,10))):
                print("soduko! row digits are not single!")
                break
            return(True)

    def checkcol(self):
        for i in range(1,10):
            # temp = getrow(self,i)
            # temp2 = sorted(temp)
            if (sorted(self.getcol(i)) != list(range(1,10)) ):
                print("soduko! col digits are not single !")
                break
            return(True)

    def checkbox(self):
        for i in range(0,9):
            # temp = getrow(self,i)
            # temp2 = sorted(temp)
            if (sorted(self.getbox(i)) != list(range(1,10)) ):
                print("soduko! digits are not single !")
                break
            return(True)
    
# ------------------------------------------------------------------------------------------------------------------------- #

    # verifying function : check whether a sudoku object is solved in 3 critieria
    # parameter : sudoku object
    # return : True or False

    def verify_solution(self): 
        # verifying is easy!!
        # print("Could be!  Let's just say yes!!")

        # 1. initial values 
        nonempty_init = [self.copy[x] for x in self.nonempty] # initially nonempty cell
        nonempty_sol = [self.puzzle[x] for x in self.nonempty] # nonempty cell of current state
        boo =  (sum([i - j for i, j in zip(nonempty_sol, nonempty_init)]) == 0) 
        # always true unless assign value to pre exisiting cell (bug)

        #  2. no empty cell
        boo = boo & (len(self.empty) == 0)

        # 3. row / col / box
        boo = boo and self.checkrow() and self.checkcol() and self.checkbox()

        return(boo)



                
# ------------------------------------------------------------------------------------------------------------------------- #
# -------------------- RECURSE (NOT YET CHECKED UNDER OOP SYNTAX) --------------------------------------------------------- #
# ------------------------------------------------------------------------------------------------------------------------- #            
    
    def assign_cell(self,i,cell_value):
        self.puzzle[i] = cell_value
        self.empty.remove(i)
        # remove
        self.unrow[i//9].remove(cell_value)
        self.uncol[i%9].remove(cell_value)
        self.unbox[ (i - 27*(int(i/27))) % 9 // 3  +  3*int(i/27) ].remove(cell_value)
    
    # unassign cell       
    def unassign_cell(self, i, cell_value):
        # revert
        self.puzzle[i] = 0 # empty again
        self.empty.append(i) # empty index
        self.unrow[i//9].append(cell_value)
        self.uncol[i%9].append(cell_value)    
        self.unbox[ (i - 27*(int(i/27))) % 9 // 3  +  3*int(i/27) ].append(cell_value)
        pass
    
    # recurse
    def recurse(self):
    
        # Loop over all the cells.
        for cell in range(81):

            # if it's assigned, keep going
            if self.puzzle[cell]: continue

            for poss in self.cell_candidate(cell):

                # Assign the cell
                self.assign_cell(cell, poss)

                # continue deeper in the recursion.
                if self.recurse(): return True

                # If this choice failed -- 
                # at some point there were no options,
                # then we have a contradiction.
                # Unassign the cell and revert the 
                # possibilities, and try the next one.
                self.unassign_cell(cell,poss)

        # When there is no possible value,
        # we have a contradiction, 
        # and use unassign to back up.
        return False

        # until we fall off the end.
        return self.verify_solution()
    
# ------------------------------------------------------------------------------------------------------------------------- #
# -------------------- MORE STRATEGY -------------------------------------------------------------------------------------- #
# ------------------------------------------------------------------------------------------------------------------------- #

    # undate unassigned fields
    
    def update(self):
        for i in range(9):
            self.unrow[i] = [x for x in list(range(1,10)) if x not in self.getrow(i)]

        for i in range(9):
            self.uncol[i] = [x for x in list(range(1,10)) if x not in self.getcol(i)]

        for i in range(9):
            self.unbox[i] = [x for x in list(range(1,10)) if x not in self.getbox(i)]

        for i in range(9):
            for j in range(9):
                idx = 9*i+j
                if (self.puzzle[idx]): # cell index pre occupied
                    pass
                else:
                    k = (idx - 27*(int(idx/27))) % 9 // 3  +  3*int(idx/27)
                    self.uncell[idx] = list(set.intersection(set(self.unrow[i]), set(self.uncol[j]), set(self.unbox[k])))
        pass

# ------------------------------------------------------------------------------------------------------------------------- #
    # assign "hidden" single cell
    
    def assign_hidden(self, idx):
        # compare cell[i] and the union of rest of the cell in same row / col
        # for all i.j in {row / col index}, i != j 
        # unassigned list of cell[i] - union( unassigned list of cell[j] )
        for i in idx: 
            idx.remove(i)
            t = idx
            union = set([])
            for j in t:
                union = set.union(set(union), set(self.uncell[j]))
            idx.append(i)
            idx.sort()
            hidden = set(self.uncell[i]) - set(union)
            pair = list(hidden)

            if list(pair):
                # print(i, pair)
                self.puzzle[i] = pair[0] # assign
                del self.uncell[i] # remove unassigned cell
                self.empty.remove(i) # remove empty index
                self.update() # update unrow uncol unbox uncell
                break
                
# ------------------------------------------------------------------------------------------------------------------------- #
    # find hidden single in row / col / box

    def hidden_row(self):
        for nrow in range(9): # traverse each row
            index_row = list(range(nrow*9,(nrow+1)*9)) # all cell index of the row
            idx = [x for x in index_row if x in self.empty] # subset non empty cells to check
            self.assign_hidden(idx) # assign hidden cell of the row
        pass

    def hidden_col(self):
        for ncol in range(9): # traverse each col
            index_col = [9*x+ncol for x in range(9)] # all cell index of the col
            idx = [x for x in index_col if x in self.empty] # subset non empty cells to check
            self.assign_hidden(idx) # assign hidden cell
        pass

    def hidden_box(self):
        for nbox in range(9): # traverse each box
            index_box = [(x%3) + (x//3)*9 + (nbox%3) * 3 + (nbox//3)*27 for x in range(9)] # all cell index of the box
            idx = [x for x in index_box if x in self.empty] # subset non empty cells to check
            self.assign_hidden(idx) # assign hidden cell
        pass
    
# ------------------------------------------------------------------------------------------------------------------------- #
    
    # assign naked single i.e. cells with only candidate
    
    def naked_single(self):
        for i in self.empty:
            if len(self.uncell[i]) == 1 :
                self.puzzle[i] = self.uncell[i][0]
                del self.uncell[i]
                self.empty.remove(i) # remove empty index
                self.update()
                
# ------------------------------------------------------------------------------------------------------------------------- #

    def solve2(self):
        # do while loop
        init = len(self.empty)
        self.hidden_row()
        self.hidden_col()
        self.hidden_box()
        self.naked_single()
        
        while (init - len(self.empty) != 0): # while converge
            init = len(self.empty)
            self.hidden_row()
            self.hidden_col()
            self.hidden_box()
            self.naked_single()
        
        
        if (self.verify_solution() == True) :
            # print("strategy 2 solved!")
            pass
        else:
            print("meh, can't be solved by hidden singles")
            
        pass
    
    
# ------------------------------------------------------------------------------------------------------------------------- #
# ## attempted pointed pair 
# ## GGWP
# nbox = 2
# index_box = [(x%3) + (x//3)*9 + (nbox%3) * 3 + (nbox//3)*27 for x in range(9)] # all cell index of the box
# idx = [x for x in index_box if x in temp.empty] # subset non empty cells to check

# for i in idx:
#     if len(temp.uncell[i]) == 2:
#         for j in idx:
#             if (set(temp.uncell[i]).issubset(set(temp.uncell[j]))) and (j!=i):
#                 rm = temp.uncell[i]
                
#                 idx.remove(i)
#                 idx.remove(j)
#                 tt = idx
                
#                 for t in tt:
#                     print(i,j, t)
#                     temp.uncell[t] = list(set(temp.uncell[t]) - set(temp.uncell[i]))

In [3]:
easy = sudoku("001700509573024106800501002700295018009400305652800007465080071000159004908007053")
easy.solve()

In [4]:
print(str(easy))


+---+---+---+
|241|768|539|
|573|924|186|
|896|531|742|
|734|295|618|
|189|476|325|
|652|813|497|
|465|382|971|
|327|159|864|
|918|647|253|
+---+---+---+


In [5]:
easy = sudoku("001700509573024106800501002700295018009400305652800007465080071000159004908007053")
easy.solve2()

In [6]:
print(str(easy))


+---+---+---+
|241|768|539|
|573|924|186|
|896|531|742|
|734|295|618|
|189|476|325|
|652|813|497|
|465|382|971|
|327|159|864|
|918|647|253|
+---+---+---+


In [7]:
solved, total = 0, 0

with open("sudoku_solved_easier.txt", "w") as out:
    for line in open("sudoku_easier.txt"):
    
        p = line.strip()
        s = sudoku(p)
        s.solve()
        # s.solve2() # almost 
        # s.assign()

        total += 1
        if s.verify_solution(): solved += 1

        out.write(s.__repr__())

print("Easy {}/{} = {:.3f}".format(solved, total, solved/total))

Easy 1000/1000 = 1.000


In [17]:
# "hidden pair" test case
test = sudoku("090010302601000000020800091008000007400008003000031850900002000002005049305490020")
print(str(test))


+---+---+---+
|090|010|302|
|601|000|000|
|020|800|091|
|008|000|007|
|400|008|003|
|000|031|850|
|900|002|000|
|002|005|049|
|305|490|020|
+---+---+---+


In [18]:
test.solve2()

meh, can't be solved by hidden singles


In [19]:
print(str(test))


+---+---+---+
|090|010|302|
|601|923|000|
|023|800|091|
|038|049|207|
|400|208|903|
|209|031|854|
|900|102|030|
|002|305|049|
|305|490|020|
+---+---+---+


In [ ]:
test.recurse()

In [21]:
print(str(test)) # recurse interrupted


+---+---+---+
|894|516|372|
|651|923|480|
|723|804|591|
|138|649|207|
|467|258|913|
|209|731|854|
|986|102|735|
|072|385|649|
|305|497|128|
+---+---+---+
